# HOTP

## ✅ 이벤트 동기화 방식 = HOTP (HMAC-based One-Time Password)

> 시간과 무관하게 “몇 번째 시도인지”를 기준으로 OTP 생성
>
>
> → 즉, **카운터 기반**
>

---

## 🔸 공식

```
HOTP = Truncate(HMAC(K, counter))
```

- `K`: 비밀 키 (시크릿 키)
- `counter`: 시도 횟수, 이벤트 번호
- 클라이언트와 서버는 이 **counter 값을 동기화**해야 함

---

## ✅ 예제 코드 (Python)

In [1]:
import pyotp

# 시크릿 키 (양쪽에 동일해야 함)
secret = pyotp.random_base32()

# 이벤트 카운터 기반 HOTP 객체
hotp = pyotp.HOTP(secret)

# 예: 사용자가 5번째 요청이면
otp = hotp.at(5)

print("5번째 OTP:", otp)

# 검증 (서버 측)
is_valid = hotp.verify(otp, 5)
print("검증 결과:", is_valid)

5번째 OTP: 211740
검증 결과: True


---

## ✅ 실무자용 요약

| 항목 | 설명 |
| --- | --- |
| 동기화 대상 | 서버와 클라이언트의 **카운터 값** |
| 단점 | 클라이언트가 버튼 여러 번 누르면 카운터 밀림 → **동기화 필요** |
| 보완 방법 | 서버는 몇 개 범위 허용 (`look-ahead window`) |

---

## ✅ 시험 스타일 용어 대응

| 보기 문장 | 해설 |
| --- | --- |
| “OTP는 특정 이벤트 발생 시마다 새로운 비밀번호를 생성한다.” | → ✅ HOTP 설명 |
| “OTP는 현재 시간에 따라 비밀번호가 생성된다.” | → ✅ TOTP 설명 |
| “OTP는 난수 기반으로 생성된다.” | → ❌ HMAC 기반, 난수 아님 |

---

## ✅ HOTP vs TOTP 비교

| 항목 | HOTP (이벤트 동기화) | TOTP (시간 동기화) |
| --- | --- | --- |
| 입력값 | HMAC(K, **counter**) | HMAC(K, **timestamp//30**) |
| 동기화 | 카운터 | 시간 |
| 실무 사용 | 드묾 (하드웨어 토큰) | 일반적 (구글 OTP 등) |
| 시간 필요 | ❌ | ✅ |
| 누르면 새로고침 | ✅ (버튼 누를 때마다) | ❌ (30초마다 자동 변경) |

---

## ✅ 한줄 요약

> 🔸 이벤트 동기화 = HOTP = HMAC(K, counter)
>
>
> 🔸 클라이언트와 서버가 **몇 번째 요청인지 숫자(counter)를 동기화**해야 함
>
> 🔸 pyotp에서는 `.at(n)`으로 생성, `.verify()`로 검증
>

---

# "시간 + 이벤트" 동시 기반 OTP

## ✅ 시간 + 이벤트 기반 OTP: 이론적 확장

| 구분 | 설명 |
| --- | --- |
| 🔸 시간 기반 (TOTP) | HMAC(secret, timestamp / 30) |
| 🔸 이벤트 기반 (HOTP) | HMAC(secret, counter) |
| 🔸 **혼합 기반 (복합형)** | HMAC(secret, timestamp ⊕ counter) 또는 복합 입력 |

---

## ✅ 예시 공식

```
OTP = Truncate(HMAC(K, f(counter, timestamp)))
     = HMAC(K, counter + currentTimeBlock)
```

- `f()`는 단순히 두 값을 이어붙이거나 XOR, 또는 Hash 할 수도 있음
- 이벤트가 발생한 시간 기준으로 OTP 생성 → **시간 위조 + 재전송 공격 모두 차단 가능**

---

## ✅ 실무에서 사용되는 예?

- **보안 강도 강화용 커스텀 OTP**
    - 고위험 금융 시스템 등에서 사용 가능성 있음
    - IoT 디바이스 일부는 타임스탬프 + 센서 이벤트로 OTP 유사한 인증을 하기도 함
- **FIDO/OTP 기반 2FA**에서 변형 가능

---

## ✅ 시험 보기 예시 대응

| 보기 | 해설 |
| --- | --- |
| “OTP는 시간과 이벤트 모두를 고려해 생성할 수 있다” | ⭕ |

| “OTP는 반드시 하나의 입력만을 기반으로 한다” | ❌ HOTP or TOTP만 본 보기 (혼합 무시함) |

| “혼합형 OTP는 이벤트 횟수 및 시간 동기화가 모두 필요하다” | ⭕

---

## ✅ 정리: 실무자 감각용

> 🔹 일반적으로는 HOTP(TOTP) 단일 방식 사용
>
>
> 🔹 하지만 고급 보안 요구 시 **시간 + 이벤트 동시 기반도 구현 가능**
>
> 🔹 시험에서도 “혼합 기반도 존재”하는 것으로 인정됨
>